In [1]:
import pandas as pd
import numpy as np

In [2]:
ti = pd.read_csv("tb_dados_pacientes40.csv", sep=';', nrows=100)
ti["sexo"] = ti["sexo"].apply(lambda x: 'M' if x == 1 else 'F')
ti['cpf'] = ti['cpf'].astype('Int64').astype('str').replace({'<NA>': ""})
ti['cns'] = ti['cns'].astype('Int64').astype('str').replace({'<NA>': ""})
ti['cep'] = ti['cep'].astype('Int64').astype('str').replace({'<NA>': ""})
#ti = ti.fillna('')
ti.replace('', np.nan, inplace=True)
ti = ti.dropna()
ti

,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep
3,TEREZINHA,FERREIRA,LOPES,LUIZA,JESUS,F,1939-03-01,33685053604,107953622820018,GUANHAES_MG,35740000
31,VANDENIR,SILVA,ASSUNCAO,MARLUSE,ASSUNCAO,F,1975-08-21,7151872798,898001176669739,ALEM PARAIBA_MG,36660000
45,MARIA,EFIJENIA RIBEIRO,SAN,MARIA,RIBEIRO,F,1954-08-26,68354312615,801434168311037,BELO HORIZONTE_MG,32450000
47,CARLOS,ROBERTO,CASTRO,APARESIDA,CASTRO,M,1970-10-28,451013611,898050011070791,FORMIGA_MG,35570000
87,SUELI,APARESIDA COSTA,SANTOS,MARIA,BORJES,F,1971-06-01,6984474680,898001174534232,VARGINHA_MG,37002140


In [3]:
from phe import paillier
pk, sk = paillier.generate_paillier_keypair()

In [4]:
class Cliente:
    def __init__(self, pk, sk): # método construtor da classe
        self.pk = pk
        self.sk = sk

    def elege_representantes(self, ti, colunas):
        rr = ti.drop(ti.index) # cria conjunto de dados vazio com mesma estrutura de colunas que ti
        groups = ti.groupby(colunas).size().reset_index(name='counts').sort_values('counts', ascending=False)
        ti['bloco'] = 0

        b = 0
        for _, row in groups.iterrows():
            conditions = []
            for c in colunas:
                conditions.append(ti[c] == row[c])

            filtered = ti.loc[np.logical_and.reduce(conditions)]
            ti.loc[np.logical_and.reduce(conditions), ['bloco']] = b
            b += 1
            
            
            data = {}
            for col in filtered.loc[:, filtered.columns != 'bloco'].columns:
                mc = filtered.groupby([col]).size().reset_index(name='counts').sort_values('counts', ascending=False).head(1).reset_index()
                data[col] = [mc.at[0, col]]

        
            d = pd.DataFrame.from_dict(data)
            rr = pd.concat([rr, d])
        rr = rr.drop(['bloco'], axis=1, errors='ignore')
        return ti, rr.reset_index(drop=True).reset_index(names=['bloco'])


    def encriptar(self, dados):
        e_dados = dados.copy(deep=True)
        for i, row in e_dados.loc[:, e_dados.columns != 'bloco'].iterrows():
            for c, item in row.items():
                bin_word = []
                for w in str(item):
                    bin_word.append(ord(w))
                e_dados.at[i, c] = [self.pk.encrypt(x) for x in bin_word]
        return e_dados

    def escore(self, es):
        s = es.copy(deep=True)
        s = self.__desencriptar(s)
        
        s['pontuacao'] = 0
        for i, row in s.iterrows():
            for column_name, word in row.items():
                if word is None or column_name in ['bloco', 'pontuacao']:
                    continue
                if all(letter == 0 for letter in word):
                    s.at[i, 'pontuacao'] += 1
        return s
        

    def __desencriptar(self, es):
        s = es.copy(deep=True)
        for i, row in es.loc[:, es.columns != 'bloco'].iterrows():
            for column_name, word in row.items():
                if word is None:
                    continue
                s.at[i, column_name] = [sk.decrypt(x) for x in word]
        return s

In [5]:
bob = Cliente(pk, sk)
ti, rr = bob.elege_representantes(ti, ['nomeu'])

In [6]:
eti = bob.encriptar(ti)
eti

,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep,bloco
3,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,2
31,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,0
45,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,3
47,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,1
87,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,4


In [7]:
err = bob.encriptar(rr)
err

,bloco,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep
0,0,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...
1,1,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...
2,2,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...
3,3,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...
4,4,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...


In [8]:
class Servidor:
    def __init__(self, eti, err, pk): # método construtor da classe
        self.eti = eti
        self.err = err
        self.pk = pk

    def subtracao_homomorfica(self, etc):
        esrr = self.err.copy(deep=True)
        
        for i, row in self.err.iterrows():
            for column, item in row.items():
                if column == 'bloco':
                    esrr.at[i, column] = item
                    continue
                    
                if len(item) != len(etc.at[0, column]):
                    esrr.at[i, column] = None
                    continue

                esrr.at[i, column] = []
                li = 0
                for letter in item:
                    esrr.at[i, column].append(item[li] - etc.at[0, column][li]) # subtração homomórfica
                    li += 1
        return esrr


    def subtracao_homomorfica_eti(self, etc, blocos):
        bloco_dados = self.eti[self.eti['bloco'].isin(blocos)]
        esti = bloco_dados.copy(deep=True)
        
        for i, row in bloco_dados.iterrows():
            for column, item in row.items():
                if column == 'bloco':
                    esti.at[i, column] = item
                    continue
                    
                if len(item) != len(etc.at[0, column]):
                    esti.at[i, column] = None
                    continue

                esti.at[i, column] = []
                li = 0
                for letter in item:
                    esti.at[i, column].append(item[li] - etc.at[0, column][li]) # subtração homomórfica
                    li += 1
        return esti
        

In [9]:
alice = Servidor(eti, err, pk)

In [10]:
carol = Cliente(pk, sk)

In [11]:
tc = pd.DataFrame({
    'nomep': ['VANDENIR'],
    'nomem': ['SILVA'],
    'nomeu': ['ASSUNCAO'],
    'nomemaep': ['MARLUSE'],
    'nomemaem': ['ASSUNCAO'],
    'sexo': ['F'],
    'data': ['1975-08-21'],
    'cpf': ['7151872798'],
    'cns': ['898001176669739'],
    'codmunicipio': ['ALEM PARAIBA_MG'],
    'cep': ['36660000'],
})			
tc

,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep
0,VANDENIR,SILVA,ASSUNCAO,MARLUSE,ASSUNCAO,F,1975-08-21,7151872798,898001176669739,ALEM PARAIBA_MG,36660000


In [12]:
etc = carol.encriptar(tc)
etc

,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep
0,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...


In [13]:
esrr = alice.subtracao_homomorfica(etc)
esrr

,bloco,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep
0,0,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...
1,1,None,None,None,None,None,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,None,[<phe.paillier.EncryptedNumber object at 0x124...,None,[<phe.paillier.EncryptedNumber object at 0x124...
2,2,None,None,None,None,None,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,None,[<phe.paillier.EncryptedNumber object at 0x124...,None,[<phe.paillier.EncryptedNumber object at 0x124...
3,3,None,None,None,None,None,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,None,[<phe.paillier.EncryptedNumber object at 0x124...,None,[<phe.paillier.EncryptedNumber object at 0x124...
4,4,None,None,None,None,None,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,None,[<phe.paillier.EncryptedNumber object at 0x124...


In [14]:
srr = carol.escore(esrr)
srr

,bloco,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep,pontuacao
0,0,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",[0],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",11
1,1,None,None,None,None,None,[7],"[0, 0, 0, -5, 0, 1, -8, 0, 0, 7]",None,"[0, 0, 0, 0, 5, -1, -1, -6, -5, -6, 1, -9, 0, ...",None,"[0, -1, -1, 1, 0, 0, 0, 0]",0
2,2,None,None,None,None,None,[0],"[0, 0, -4, 4, 0, 0, -5, 0, -2, 0]",None,"[-7, -9, -1, 9, 5, 2, 5, -5, -4, 2, -4, -9, -7...",None,"[0, -1, 1, -2, 0, 0, 0, 0]",1
3,3,None,None,None,None,None,[0],"[0, 0, -2, -1, 0, 0, 0, 0, 0, 5]",None,"[0, -9, -7, 4, 3, 3, 0, -1, 2, -3, -5, -8, -7,...",None,"[0, -4, -2, -1, 0, 0, 0, 0]",1
4,4,None,None,None,None,None,[0],"[0, 0, 0, -4, 0, 0, -2, 0, -2, 0]","[-1, 8, 3, 3, -4, 0, 2, -1, -1, -8]","[0, 0, 0, 0, 0, 0, 0, 0, -2, -1, -3, -5, -5, 0...",None,"[0, 1, -6, -6, 2, 1, 4, 0]",1


In [15]:
esti = alice.subtracao_homomorfica_eti(etc, 
    srr.loc[srr['pontuacao'].values == srr['pontuacao'].values.max(), 'bloco'].tolist()
)
esti

,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep,bloco
31,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,[<phe.paillier.EncryptedNumber object at 0x124...,0


In [16]:
sti = carol.escore(esti)
sti

,nomep,nomem,nomeu,nomemaep,nomemaem,sexo,data,cpf,cns,codmunicipio,cep,bloco,pontuacao
31,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",[0],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",0,11


In [17]:
sti.loc[sti['pontuacao'].values == sti['pontuacao'].values.max()].index.tolist()

[31]